In [1]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
from scipy import stats

mort = "./2012-2016 CardioVasc Mortality Rates.csv"
edu = "./Education.csv"
econo = "./SAIPESNC.csv"

In [2]:
mortality_file = pd.read_csv(mort)
education_file = pd.read_csv(edu)
socio_econo_file = pd.read_csv(econo)

print(mortality_file.columns)
print()
print(education_file.columns)
print()
print(socio_econo_file.columns)

Index(['Notes', 'State', 'State Code', 'County', 'County Code', 'Deaths',
       'Population', 'Crude Rate', 'Crude Rate Lower 95% Confidence Interval',
       'Crude Rate Upper 95% Confidence Interval',
       'Crude Rate Standard Error'],
      dtype='object')

Index(['FIPS Code', 'State', 'Area name',
       'Less than a high school diploma, 2012-2016',
       'High school diploma only, 2012-2016',
       'Some college or associate's degree, 2012-2016',
       'Bachelor's degree or higher, 2012-2016',
       'Percent of adults with less than a high school diploma, 2012-2016',
       'Percent of adults with a high school diploma only, 2012-2016',
       'Percent of adults completing some college or associate's degree, 2012-2016',
       'Percent of adults with a bachelor's degree or higher, 2012-2016'],
      dtype='object')

Index(['State', 'County ID', 'State / County Name',
       'All Ages SAIPE Poverty Universe', 'All Ages in Poverty Count',
       'All Ages in Poverty Count LB 

In [3]:
education_file = education_file.rename(columns={'FIPS Code':'County ID'})
education_file.head()

,County ID,State,Area name,"Less than a high school diploma, 2012-2016","High school diploma only, 2012-2016","Some college or associate's degree, 2012-2016","Bachelor's degree or higher, 2012-2016","Percent of adults with less than a high school diploma, 2012-2016","Percent of adults with a high school diploma only, 2012-2016","Percent of adults completing some college or associate's degree, 2012-2016","Percent of adults with a bachelor's degree or higher, 2012-2016"
0,0,US,United States,"27,818,380","58,820,411","62,242,569","64,767,787",13.0,27.5,29.1,30.3
1,1001,AL,Autauga County,"4,528","12,519","10,451","8,968",12.4,34.3,28.7,24.6
2,1003,AL,Baldwin County,"13,956","40,154","44,486","41,350",10.0,28.7,31.8,29.5
3,1005,AL,Barbour County,"4,824","6,422","4,775","2,366",26.2,34.9,26.0,12.9
4,1007,AL,Bibb County,"3,040","6,586","4,234","1,890",19.3,41.8,26.9,12.0


In [4]:
new_edu_columns = ['County ID', 'State', "County Name", '< High School', 'High School', 'Associate',
                  'Bachelors <', 'Perc < High School', 'Perc High School', 'Perc Associate',
                  'Perc Bachelors <']
education_file.columns = new_edu_columns

In [5]:
edu_cleaned_columns = ['County ID', "County Name", 'Perc < High School', 'Perc High School', 'Perc Associate', 'Perc Bachelors <']
education_clean_df = education_file[edu_cleaned_columns]
education_clean_df['MYS'] = ""
education_clean_df.head()

C:\Users\corey\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,County ID,County Name,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS
0,0,United States,13.0,27.5,29.1,30.3,
1,1001,Autauga County,12.4,34.3,28.7,24.6,
2,1003,Baldwin County,10.0,28.7,31.8,29.5,
3,1005,Barbour County,26.2,34.9,26.0,12.9,
4,1007,Bibb County,19.3,41.8,26.9,12.0,


In [6]:
mean_years_schooling = []
for index, row in education_clean_df.iterrows():
    mean_years_schooling.append(((row['Perc < High School']*10) + (row['Perc High School'] * 12) + (row['Perc Associate'] * 14)
                              + (row['Perc Bachelors <'] * 20))/100)
    
education_clean_df['MYS'] = mean_years_schooling
education_clean_df['MYS'].describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

C:\Users\corey\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    3143.000000
mean       13.989138
std         0.834010
min        12.010000
10%        13.038400
20%        13.280800
30%        13.502000
40%        13.706000
50%        13.882000
60%        14.070000
70%        14.290800
80%        14.603200
90%        15.104800
max        18.602000
Name: MYS, dtype: float64

In [13]:
education_clean_df['Education Bin'] = ""

education_bins = [12, 13.05, 13.28, 13.502, 13.706, 13.882, 14.07, 14.291, 14.6032, 15.1048, 18.603]
education_bin_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
education_clean_df['Education Bin'] = pd.cut(education_clean_df['MYS'], bins=education_bins, labels=education_bin_labels)
education_clean_df.head()

C:\Users\corey\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\corey\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,County ID,County Name,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS,Education Bin
0,0,United States,13.0,27.5,29.1,30.3,14.734,9
1,1001,Autauga County,12.4,34.3,28.7,24.6,14.294,8
2,1003,Baldwin County,10.0,28.7,31.8,29.5,14.796,9
3,1005,Barbour County,26.2,34.9,26.0,12.9,13.028,1
4,1007,Bibb County,19.3,41.8,26.9,12.0,13.112,2


In [8]:
socio_columns = ['State', 'County ID', 'County Name', '1', '2', '3', '4',' 5', '6','7', 'Median Household Income', '8', '9']
socio_econo_file.columns = socio_columns
socio_econo_file.head()

,State,County ID,County Name,1,2,3,4,5,6,7,Median Household Income,8,9
0,0,0,United States,310764925,47240290,46990033,47490547,15.2,15.1,15.3,54134,54052,54216
1,1,1001,Autauga County (AL),54905,7206,5914,8497,13.1,10.8,15.5,53748,49752,57745
2,1,1003,Baldwin County (AL),197091,25850,21888,29811,13.1,11.1,15.1,50976,48035,53917
3,1,1005,Barbour County (AL),23311,6731,5511,7952,28.9,23.6,34.1,32111,29316,34907
4,1,1007,Bibb County (AL),20316,4268,3359,5177,21.0,16.5,25.5,40113,36439,43787


In [9]:
# Change county name to just have county name, not state name, too.
socio_cleaned_columns = ['County ID', 'County Name', 'Median Household Income']
socio_econo_clean_df = socio_econo_file[socio_cleaned_columns]
socio_econo_clean_df.head()

,County ID,County Name,Median Household Income
0,0,United States,54134
1,1001,Autauga County (AL),53748
2,1003,Baldwin County (AL),50976
3,1005,Barbour County (AL),32111
4,1007,Bibb County (AL),40113
